In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint) # tokenizer from checkpoint
model = AutoModelForSequenceClassification.from_pretrained(checkpoint) # model from checkpoint

In [6]:
sequence = "I've been waiting for a HuggingFace course my whole life"

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids]) # here we add a dimension by passing our ids in an array (this creates a batch of ids, in this case, the batch size is 1)
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166]])
Logits: tensor([[-3.1398,  3.3515]], grad_fn=<AddmmBackward0>)
